<img src='https://user-images.githubusercontent.com/6457691/90080969-0f758d00-dd47-11ea-8191-fa12fd2054a7.png' width = '200' align = 'right'>

## *DATA SCIENCE / SECTION 4 / SPRINT 3 / Assignment 1*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model 을 사용해보기
- <a href="#p2">Part 2:</a> Custom CNN Model을 제작해보기
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


케라스를 이용한 바이너리 이미지 분류 모델에 3가지 CNN 모델을 적용한다.  <br>
[데이터 다운로드](https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/mountainForest.zip) <br>
산의 이미지(./data/mountin/*)와 숲의 이미지(./data/forest/*)를 분류하십시오. 산을 포스티브 클래스(1)로, 숲 이미지를 네거티브(0)로 처리한다.

|Mountain (+)|Forest (-)|
|---|---|
|![](https://github.com/codestates/ds-cs-section4-sprint3/blob/main/N431/data/mountain/art1131.jpg?raw=1)|![](https://github.com/codestates/ds-cs-section4-sprint3/blob/main/N431/data/forest/cdmc317.jpg?raw=1)|

표본이 작다는 점을 감안하면 문제는 현실적으로 어렵다. 즉, 클래스당 약 350개의 관측치가 있다. 이 샘플 크기는 직장에서 이미지 분류 문제/솔루션을 프로토타이핑할 때 기대할 수 있는 것일 수 있다. 이럴 때에는 여러가지 모델을 적용해보는 것이 중요하기 때문에 이번 sprint에서는 가능한 여러 모델을 평가하는 데 익숙해지는 것이 중요합니다.

### 문항 1) CNN을 설명한 문장에서 틀린 것을 모두 고르시오. 
1. CNN은 이미지 처리에 탁월한 성능을 보이는 신경망이다. 
2. CNN의 핵심 모듈은 합성곱(convolution)과 pooling으로 기존신경망에서 사용하지 않던 개념이 새롭게 도입되었다. 
3. Weight sharing 개념이 도입되었다.
4. 이미지의 RGB처럼 3개의 채널의 정보를 이용할 수 있고, 각각의 채널에서 가중치를 곱하여 특징의 숫자를 늘리는 등 채널을 변형하여 더 많은 채널로 특징을 추출한다.  
5. 슬라이딩 윈도우 방식으로 이미지에 filter들을 곱하는 형태로 적용한다. 



### 문항 2) CNN을 설명한 문장에서 틀린 것을 모두 고르시오. 
6. 합성곱연산을 통해 나온 결과를 특성 맵(Feature map)이라 한다. 
7. filter의 개수와 맞춰서 연산을하고, 합성곱 이후에도 이미지의 사이즈를 유지하려고 패딩(padding)이란 기술을 사용한다. 
8. 가중치와 편향의 계산법은 일반 신경망과 동일하다.
9. stride가 1보다 크면 합성곱을 통과하더라도 이미지의 사이즈가 일정하다. 
10. 3차원 convolution도 수행할 수 있다. 

# Part 1

## Pre - Trained Model
<a id="p1"></a>

Keras에서 제공하는 pre-trained 모델인 ResNet50을 불러오고, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - 이 모델은 50 개의 layer를 가진  CNN기반의 모델입니다. 총 [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt)를 분류하는 모델로 아래와 같이 사용할 수 있습니다. 우리가 풀어야 할 과제는 2가지 이니, 마지막 출력 단을 변경해서 사용하면 됩니다.

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D()
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

`ResNet50`을 불러올 때, `include_top`을 False로 하면, 기존 1000개의 분류 문제를 풀 수 있는 ResNet 모델에서 Fully Connected layer를 제거해주는 역할을 합니다. 

```python
for layer in resnet.layers:
    layer.trainable = False
```
이 부분은 ResNet50 레이어들의 파라미터 학습모드를 끄는(off) 것입니다. 이렇게 설정된 매개 변수는 설령 지금 순전파/역전파 이후 오류가 전파 되더라도 파라미터가 학습(업데이트)되지 않는 것입니다. 

Keras 기능 API를 사용하여 모델에 추가로 `Fully-conneted layer`(Dense모델)을 추가해야합니다. 우리는 최상위 레이어,  `Fully-conneted layer`를 제거합니다. 출력층의 숫자(=분류할 클래스의 개수)가 맞지 않기 때문이죠. 네트워크의 특징을 추출하는 부분 만 유지하는 것입니다. `GlobalAveragePooling2D` 레이어는 마지막 컨벌루션 레이어 출력 (2 차원) 각각의 평균을 취함으로써 정말 멋진 평탄화 기능으로 작동합니다.

```python
x = res.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x) # 출력층 설계
model = Model(res.input, predictions)
```

이 과제는 산의 이미지 (`. / data / mountain / *`)와 숲의 이미지 (`. / data / forest / *`)를 분류하기 위해 위의 전이 학습(transfer learning)을 적용하는 것입니다. 산을 postive 클래스 (1)로, 숲 이미지를 음수 (0)로 취급합니다.

Steps to complete assignment: 
1. 이미지 데이터를 numpy 배열 (`X`)로 불러옵니다.
2. 레이블에 대한 `y`를 만듭니다.
3. resnet의 사전 훈련 된(pre-trained) 레이어로 모델 훈련
4. 모델의 정확성보고

### 문항 3) Cifar100같이 100개의 클래스로 Transfer learning을 해야 한다면, 아래 코드를 어떻게 바꿔야 하는가? <br> 변경된 코드를 입력하시오. 
```
predictions = Dense(1, activation='sigmoid')(x)
```



### 문항 4) 전이학습을 진행할 때, `layer.trainable = True`로 진행하는 것과 'False'일 때의 차이에 대해서 틀린 설명을 **모두** 고르시오.

- [1] True인 경우 네트워크에 있는 모든 파라미터들이 학습이 진행되어, 학습속도가 느려진다. 
- [2] True인 경우에도 학습속도가 느려지진 않는다.
- [3] False인 경우가 True인 경우보다 학습할 파라미터의 개수가 적다. 
- [4] False인 경우와 True인 경우 총 가중치 파라미터의 개수는 같다. 


## Load in Data

![skimage-logo](https://scikit-image.org/_static/img/logo.png)

기존 학습된 네트워크(모델)을 사용하려면 기존 입력값을 동일하게 맞춰주어야 합니다.  [`skimage`](https://scikit-image.org/) 의 문서를 확인하시면 보다 쉽게 사용하실 수 있을 것입니다. 다음 함수를 사용해보세요. `skimage.io.imread_collection` 와 `skimage.transform.resize`.

### 문항 5) 이미지 입력을 resize 해야하는 이유와 함수를 사용하는 방법에 대해서 틀린 것을 **모두** 고르시오. 이를 해결하기 위해서 실제 구동을 시켜보아도 된다. 
- [1] 다른 사이즈의 이미지를 입력하게 되면, 연산이 되지 않는다. 
- [2] 다른 사이즈의 이미지를 입력하게 되면 기존의 파라미터가 특징을 추출하는데 적절한 정보가 담기지 못한다. 
- [3] resize함수를 default로 이용하면, 기존 이미지를 interpolation하여 이미지를 생성한다. 
- [4] resize를 사용할 때 선택할 수 있는 모드의 개수는 총 5개이다. 
- [5] mode에 따라서 인터폴레이션이 되기도, 안 되기도 한다. 
- [6] anti_aliasing 을 이용하면 가우시안 필터를 입히는 데, 이는 다운샘플링을 할 때 artifact를 제거하는데 효과적이다. 


## Instatiate Model

/Users/jonathansokoll/anaconda3/envs/U4-S3-DNN/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


## Fit Model

Train on 561 samples, validate on 141 samples
Epoch 1/5
561/561 [==============================] - 39s 69ms/sample - loss: 0.1216 - accuracy: 0.9715 - val_loss: 1.4221 - val_accuracy: 0.0000e+00
Epoch 2/5
561/561 [==============================] - 35s 62ms/sample - loss: 0.0410 - accuracy: 0.9875 - val_loss: 2.9222 - val_accuracy: 0.0000e+00
Epoch 3/5
561/561 [==============================] - 35s 62ms/sample - loss: 0.0646 - accuracy: 0.9697 - val_loss: 0.9962 - val_accuracy: 0.0000e+00
Epoch 4/5
561/561 [==============================] - 39s 70ms/sample - loss: 0.0723 - accuracy: 0.9733 - val_loss: 0.2627 - val_accuracy: 1.0000
Epoch 5/5
561/561 [==============================] - 38s 68ms/sample - loss: 0.0333 - accuracy: 0.9893 - val_loss: 0.4859 - val_accuracy: 1.0000


# Part 2

## Custom CNN Model


이 단계에서는 Keras를 사용하여 자신 만의 CNN을 작성하고 훈련합니다. 네트워크 시작 부분에 적어도 하나의 Conv 레이어와 pooling 레이어가있는 아키텍처를 사용할 수 있습니다. 원하는 경우 더 추가 해볼 수 있습니다.

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 215, 215, 32)      9632      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 43, 43, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 39, 39, 64)        51264     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 9, 9, 64)          102464    
_________________________________________________________________
flatten_1 (Flatten)          (None, 5184)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 64)               

In [ ]:
# Compile Model


In [ ]:
# Fit Model


Train on 561 samples, validate on 141 samples
Epoch 1/5
561/561 [==============================] - 18s 32ms/sample - loss: 0.2667 - accuracy: 0.9073 - val_loss: 0.1186 - val_accuracy: 0.9858
Epoch 2/5
561/561 [==============================] - 18s 32ms/sample - loss: 0.2046 - accuracy: 0.9073 - val_loss: 0.3342 - val_accuracy: 0.8511
Epoch 3/5
561/561 [==============================] - 18s 32ms/sample - loss: 0.1778 - accuracy: 0.9287 - val_loss: 0.2746 - val_accuracy: 0.8723
Epoch 4/5
561/561 [==============================] - 18s 32ms/sample - loss: 0.1681 - accuracy: 0.9323 - val_loss: 0.8487 - val_accuracy: 0.5957
Epoch 5/5
561/561 [==============================] - 18s 32ms/sample - loss: 0.1606 - accuracy: 0.9394 - val_loss: 0.3903 - val_accuracy: 0.8582


# Part 3. 도전과제

## 도전과제1. Image Manipulations(Augmentation)을 이용하여 성능을 개선해 보시오. 

이미지 샘플의 증가를 시뮬레이션하기 위해 자르기, 회전, 늘이기 등과 같은 이미지 조작 기술을 적용 할 수 있습니다. 운 좋게도 Keras에는 이러한 기술을 산과 숲 예제에 적용 할 수있는 몇 가지 편리한 기능이 있습니다. 시작하는 데 도움이되는 다음 리소스를 확인하세요.

참고자료 
1. [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class)
2. [Building a powerful image classifier with very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
3. [Data Augmentation](https://www.tensorflow.org/tutorials/images/data_augmentation)

## 도전과제 2. 모델 개선하기
- 코드 향상시키기 : 지난 주 강의 자료를 참조하여 클래스와 내부 method를 이용하여 동작하고, 직접 다운로드 한 이미지를 인식시켜 범주(Class)를 분류하도록 코드개선합니다 (예 : 비행기, 풍선 이미지 다운로드 및 인식되는 지 확인)
- 데이터 augmentation을 이용하여 학습 및 분류 진행해보기 (단 Training Set, Test Set의 데이터가 섞이지 않도록 주의 필요)
- [기타 사용 가능한 사전 훈련 된 네트워크](https://tfhub.dev)를 확인하고 몇 가지 시도하고 비교해보세요.
- 전이 학습을 이용하되, 직접 가져온 이미지를 넣고 새로운 카테고리로 분류해보세요. [Classifier 재학습](https://www.tensorflow.org/hub/tutorials/image_retraining)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language